In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("/kaggle/input/products/products.csv", sep='\t',low_memory=False)

In [ ]:
print(data.shape)
print(data.dtypes)

In [ ]:
data.head()

In [ ]:
# 1. Fonction qui donne le % de données manquantes (NaN) du dataframe

def ratio_nan(dataframe):
        print('*** Ratio de NaN: {} %, Taille du jeu de données : {}\n'
              .format(round(dataframe.isna().sum().sum()/dataframe.shape[0]/dataframe.shape[1],2)*100,dataframe.shape))



    # 1.1 Fonction qui donne le % de données manquantes (NaN) par colonne

def ratio_nan_colonne(dataframe):
    ratio_nan_colonne = (dataframe.isna().sum()*100/(dataframe.shape[0])).sort_values(ascending=False)
    
    # Afficahge en graphique
    
    sns.set(style="dark")
    plt.figure(figsize=(17, 31))
    ax = sns.barplot(y = ratio_nan_colonne.index, x=ratio_nan_colonne.values)
    ax.xaxis.set_ticks_position('top')
    plt.title('1.1 Données manquantes par colonne (en %)', size=15)
    plt.show()

    
    
    # 1.2 Fonction qui donne le % de données manquantes (NaN) par ligne

def ratio_nan_ligne(dataframe):
    ratio_nan_ligne = pd.DataFrame(data = round(dataframe.isna().sum(axis=1)*100/dataframe.shape[1]), columns = ['%'])
    print(' 1.2 Ratio de NaN par ligne:\n')
    print(ratio_nan_ligne)

In [ ]:
# 2. Réduction du dataframe à la France

def filter_france(dataframe):
    
    france = ['France', 'FR', 'en:FR', 'en:fr', 'en:France','france', 'Francia', 'Frankreich']
    all_fr_product = []
    for c in france:
            fr_product = (data[data['countries'].str.contains(c, na=False)]).index.tolist()
            all_fr_product += fr_product

    all_fr_product = list(set(all_fr_product)) # on retire les doublons de l'index
    
    dataframe = pd.DataFrame(data=data, index = all_fr_product) 
    
    return dataframe

In [ ]:
# 3. Traitement des NaN et suppression des doublons

def nettoyage_nan(dataframe, ratio_max_nan) :
    
    # Remplacement des "faux NaN" en NaN reconnus par numpy NaN
    
    print(' ** Remplacement des faux nan et unknown en nan numpy\n')
        
    dataframe = dataframe.replace(to_replace = ['nan','NaN','unknown'], value = np.nan)
       
    ratio_nan(dataframe)
    
    # Suppression des produits sans nom ou code
    
    print(' ** Suppression des lignes sans nom ou code\n')
    dataframe = dataframe.drop(dataframe[dataframe['product_name'].isna()].index.tolist())
    dataframe = dataframe.drop(dataframe[dataframe['code'].isna()].index.tolist())
    
    ratio_nan(dataframe)
    
    # Suppression des doublons   
    
    print(' ** Suppression des duplicates\n')
    dataframe.drop_duplicates(inplace=True)
    dataframe.drop_duplicates(subset='code',inplace=True)
    
    ratio_nan(dataframe)

    
    # Suppression des NaN par colonne
    
    print('** Suppression des colonnes avec plus de ',ratio_max_nan*100, '% de nan\n')
    
    donnees_nan = dataframe.isna().sum()/dataframe.shape[0]
    
    dataframe = dataframe.drop(columns = donnees_nan[donnees_nan > ratio_max_nan].index.tolist())
    nb_nan = len(donnees_nan[donnees_nan > ratio_max_nan].index.tolist())

    print(' - Nombre de colonnes supprimées :', nb_nan,'\n')
    ratio_nan(dataframe)      
        
    
    # Suppression des NaN par ligne
    
    print(' ** Suppression des lignes avec plus de ',ratio_max_nan*100, '% de nan\n')
    
    donnees_nan2 = dataframe.isna().sum(axis=1)/dataframe.shape[1]
    
    dataframe = dataframe.drop(donnees_nan2[donnees_nan2 > ratio_max_nan].index.tolist())
    nb_nan2 = len(donnees_nan2[donnees_nan2 > ratio_max_nan].index.tolist())
    
    print(' - Nombre de lignes supprimées :', nb_nan2,'\n')
    
    
    return dataframe

In [ ]:
# 4. Suppression des Outliers


def suppression_outliers(dataframe):
    '''Suppression des valeurs:
    - les données négatives
    - les valeurs supérieures à 100 g'''
    
    
    for c in dataframe.select_dtypes(include = ['float64','int32']).columns.tolist() : 
        
        if (c[-5:] == '_100g') and ('energy' not in c ) and ('nutrition' not in c):   
        # Valeurs négatives et > 100
            dataframe = dataframe[(dataframe[c] >= 0) | dataframe[c].isna()]
            dataframe = dataframe[(dataframe[c] <= 100) | dataframe[c].isna()]

    # Seuil max de 900 kcal pour energy-kcal_100g
    dataframe = dataframe[(dataframe['energy-kcal_100g'] <= 900) | dataframe['energy-kcal_100g'].isna()]

    return dataframe

In [ ]:
# 5. traitement des tirets en de début des titres des colonnes

def nettoyage_titre(dataframe):

    columns = dataframe.columns
    colonne_traitee = []
    for column in columns:
        if column[0] == '-':
            column = column[1:]
        colonne_traitee.append(column)
    dataframe.columns = colonne_traitee
    return dataframe

In [ ]:
# Application des traitements de nettoyage au dataset entier
# Fonction de nettoyage

def nettoyage_dataset(dataframe):


    # 1. Ratio de Nan par ligne et par colonne
    
    ratio_nan_colonne(dataframe)
    ratio_nan_ligne(dataframe)
    
    # 2. Réduction du dataframe à la France
    
    print('\n',' 2. Réduction du dataframe à la France')
    dataframe = filter_france(dataframe)
    ratio_nan(dataframe)   
    
    
    # 3. Nettoyage des NaN avec seuil
    
    print('\n',' 3. Traitement des colonnes et lignes \n')
    dataframe = nettoyage_nan(dataframe, ratio_max_nan = 0.75)
    ratio_nan(dataframe)


    # 4. Outliers
    
    print('\n',' 4. Outliers\n')
    dataframe = suppression_outliers(dataframe)
    ratio_nan(dataframe)

    # 5. Nettoyage des tirets dans les titres de colonnes
    
    print('\n',' 5. Nettoyage des titres de colonnes ')
    dataframe = nettoyage_titre(dataframe) 
    ratio_nan(dataframe)
    
    print('\n',' * Nettoyage des données effectué')
           
    return dataframe

In [ ]:
data_nettoye = nettoyage_dataset(data)

In [ ]:
# Vérifier les colonnes restantes afin d'en filtrer les moins pertinentes

data_nettoye.columns

In [ ]:
# 6. Traitement des colonnes non pertinentes

def traitement_colonnes(dataframe) : 
    
    # Suppression de colonnes non pertinentes après nettoyage
    
    dataframe = dataframe.drop(columns=['categories_en', 'main_category_en','countries_en','code','url','created_t','created_datetime','last_modified_t','last_modified_datetime','quantity','nova_group','energy_100g','main_category',
                            'brands','brands_tags','categories_tags','labels_tags','countries_tags', 'states','states_tags','states_en', 'ingredients_text','labels',
                           'image_url','image_small_url','image_ingredients_url','image_ingredients_small_url','image_nutrition_url','image_nutrition_small_url',
                           'countries','ingredients_from_palm_oil_n','ingredients_that_may_be_from_palm_oil_n','nutriscore_score', 'sodium_100g'])
    
    # Réorganisation des colonnes
    
    dataframe = dataframe.reindex(columns = [ 'product_name', 'categories','pnns_groups_1', 'pnns_groups_2','labels_en','nutriscore_grade', 'nutrition-score-fr_100g','additives_n','energy-kcal_100g', 'fat_100g','saturated-fat_100g', 'carbohydrates_100g','sugars_100g', 'proteins_100g','salt_100g'])
    
    return dataframe

In [ ]:
# On vérifie la liste des catégories pnns 1 & 2 pour procéder à d' éventuels traitements
print(data_nettoye['pnns_groups_1'].unique().tolist())
print(data_nettoye['pnns_groups_2'].unique().tolist())

In [ ]:
# 7. Traitements des groupes pnns

def traitement_pnns(dataframe):
    
    # Traitement des groupes pnns 1

    dataframe = dataframe.replace(to_replace = 'sugary-snacks', value = 'Sugary snacks')
    dataframe = dataframe.replace(to_replace = 'salty-snacks', value = 'Salty snacks')
    dataframe = dataframe.replace(to_replace = 'fruits-and-vegetables', value = 'Fruits and vegetables')
    dataframe = dataframe.replace(to_replace = 'cereals-and-potatoes', value = 'Cereals and potatoes')

    # Traitement des groupes pnns 2

        # Vegetables
    dataframe = dataframe.replace(to_replace = ['vegetables','legumes','Legumes','Potatoes'], value = 'Vegetables')
        # Nuts
    dataframe = dataframe.replace(to_replace = 'nuts', value = 'Nuts')
        # Pizza
    dataframe = dataframe.replace(to_replace = 'Pizza pies and quiche', value = 'Pizza pies and quiches')
        # Cerals
    dataframe = dataframe.replace(to_replace = 'cereals', value = 'Cereals')
        # Fruits
    dataframe = dataframe.replace(to_replace = 'fruits', value = 'Fruits')

    return dataframe

In [ ]:
# 8. Imputation des données manquantes par médiane du groupe pnns 1 & 2

def imputation_nan(dataframe):
    
    var_quantitative = dataframe.select_dtypes(include = ['float64']).columns
    
    for c in var_quantitative:
    
        # Par pnns_groups_2 d'abord pour plus de finesse
        dataframe[c] =  dataframe[c].fillna(dataframe.groupby('pnns_groups_2')[c].transform('median'))
        
        # Par pnns_groups_1
        dataframe[c] =  dataframe[c].fillna(dataframe.groupby('pnns_groups_1')[c].transform('median'))
        
    return dataframe

In [ ]:
# 8.1 et 8.2 Description des données avant/apres imputation

def describe(dataframe):
    
    variables = ['proteins_100g', 'carbohydrates_100g']
    
    for c in variables:
        
        sns.set(font_scale=1)
        plt.figure(figsize = (5,4))

        sns.distplot(dataframe[c], bins=40, kde_kws={'bw':1})
        titre = 'Distribution de : ' + c
        plt.title(titre)
        plt.xlabel(c)
        plt.show()
    
    return dataframe

In [ ]:
# 9. Suppression des NaN par ligne

def suppression_NaN_ligne(dataframe):
    
    # Test : 85% de NaN correpsond à  plus de 13 cases vides sur 15, sachant que le nom du produit est obligatoire

    ligne_nan = dataframe.isna().sum(axis=1)/dataframe.shape[1]
    dataframe = dataframe.drop(ligne_nan[ligne_nan > 0.85].index.tolist())
    nb = len(ligne_nan[ligne_nan > 0.85].index.tolist())

    print(' - Nombre de lignes supprimées :', nb,'\n')
    
    
    return dataframe

In [ ]:
# 10. Taille du jeu final comparée à la taille du jeu initial

def shape_data(dataframe):
    
    taille_jeux = pd.DataFrame({'Données' : ['Initiales', 'Finales'], 'Dimensions' : [data.shape,dataframe.shape],
                           
                            'Ratio de NaN (en %)' :
                            [data.isna().sum().sum()/data.shape[0]/data.shape[1]*100,
                             dataframe.isna().sum().sum()/dataframe.shape[0] / dataframe.shape[1]*100]})
    print(taille_jeux)
    return dataframe

In [ ]:
# Second et dernier traitement

def traitement_final(dataframe):
    
    # 6. Traitement des colonnes non pertinentes
    
    print('\n',' 6. Traitement des colonnes non pertinentes\n')
    dataframe = traitement_colonnes(dataframe)
    ratio_nan(dataframe)
    
    # 7. Traitements des groupes pnns
    
    print('\n',' 7. Traitements des groupes pnns\n')
    dataframe = traitement_pnns(dataframe)
    ratio_nan(dataframe)
    
    # 8. Imputation des données manquantes
    
    print('\n',' 8. Imputation des données manquantes \n')
    print('\n',' 8.1 Description des données avant imputation \n')
    dataframe = describe(dataframe)
    dataframe = imputation_nan(dataframe)
    
    print('\n',' 8.2 Description des données apres imputation \n')
    dataframe = describe(dataframe)
    ratio_nan(dataframe)
    
    # 9. Suppression des lignes avec plus de 85% de NAN
    
    print('\n',' 9. Suppression des lignes avec plus de 85% de NAN \n')
    dataframe = suppression_NaN_ligne(dataframe)
    ratio_nan(dataframe)
    
    # 10. Taille du jeu final comparée à la taille du jeu initial

    print('\n',' 10. Taille du jeu final comparée à la taille du jeu initial\n')
    shape_data(dataframe)
    
    return dataframe

In [ ]:
dataset_final = traitement_final(data_nettoye)
dataset_final

In [ ]:
# Export du fichier nettoyé
dataset_final.to_csv('dataset_final.csv',index=False)